# VII. Веб-карта с Folium (II)


## и данные портала открытых данных Правительства Москвы ...


На этой неделе мы посмотрим, как выгружать геоданные с Портала Открытых данных Правительства Москвы, а также визуализировать на интерактивной карте точки с фотографиями


## Импортируем библиотеки


In [41]:
import geopandas as gpd
import requests
import pandas as pd

import folium 
import base64
import os

## 1. DataMos API: никогда не было проще скачивать данные с data_mos...


#### 1.1 Определем параметры для запроса


In [34]:
datamos_api = '31767935-5ac6-45e0-8bb4-35238f7526a7'
data_set = 62883 # number of a dataSet (2263) ( 530 613 / 614)

url_data = f'https://apidata.mos.ru/v1/datasets/{data_set}/features?api_key={datamos_api}'

#### 1.2 Отправляем запрос


In [35]:
data_mos = requests.get(url = url_data )

смотрим на статус запроса


In [36]:
data_mos.status_code

200

смотрим на ответ


In [37]:
data_mos.json()

{'features': [{'geometry': {'coordinates': [37.900526, 55.414307],
    'type': 'Point'},
   'properties': {'datasetId': 62883,
    'rowId': None,
    'attributes': {'is_deleted': 0,
     'ID': 1,
     'Name': 'Домодедово',
     'Aeroexpress': 'есть',
     'AeroexpressStation': {'global_id': 1508979300,
      'value': 'Аэропорт Домодедово'},
     'AeroexpressTerminal': 'есть',
     'AeroexpressTicketWindow': 'есть',
     'AeroexpressTicketAutomat': 'есть',
     'AeroexpressWorkingHours': '05:30 - 23:30 (с Павелецкого вокзала), 06:00 - 23:30 (из аэропорта)',
     'MCDStation': [],
     'RailwayStation': [],
     'Latitude_WGS84': '55.414307',
     'Longitude_WGS84': '37.900526',
     'global_id': 1272818040},
    'releaseNumber': 41,
    'versionNumber': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [37.415713, 55.966771], 'type': 'Point'},
   'properties': {'datasetId': 62883,
    'rowId': None,
    'attributes': {'is_deleted': 0,
     'ID': 2,
     'Name': 'Шереметьево',
  

#### 1.3. Создаем GeoDataFrame


In [38]:
dataGDF = gpd.GeoDataFrame.from_features(data_mos.json()["features"], crs="EPSG:4326")
dataGDF.head()

,geometry,datasetId,rowId,attributes,releaseNumber,versionNumber
0,POINT (37.90053 55.41431),62883,None,"{'is_deleted': 0, 'ID': 1, 'Name': 'Домодедово...",41,1
1,POINT (37.41571 55.96677),62883,None,"{'is_deleted': 0, 'ID': 2, 'Name': 'Шереметьев...",41,1
2,POINT (37.28629 55.60506),62883,None,"{'is_deleted': 0, 'ID': 3, 'Name': 'Внуково', ...",41,1
3,POINT (37.51082 55.50286),62883,None,"{'is_deleted': 0, 'ID': 4, 'Name': 'Остафьево'...",41,1
4,POINT (38.11781 55.56155),62883,None,"{'is_deleted': 0, 'ID': 5, 'Name': 'Жуковский'...",41,1


#### 1.4 Разбиваем атрибуты на разные поля


In [39]:
dataGDF_attributes = pd.DataFrame(dataGDF['attributes'].values.tolist(), index=dataGDF.index).applymap(str)
dataGDF_final = pd.concat([dataGDF, dataGDF_attributes], axis = 1).drop('attributes', axis = 1)

dataGDF_final.head()


/var/folders/ry/9bb7wrz54vq_kn2ytlj6ynzm0000gn/T/ipykernel_39923/1609880950.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dataGDF_attributes = pd.DataFrame(dataGDF['attributes'].values.tolist(), index=dataGDF.index).applymap(str)


,geometry,datasetId,rowId,releaseNumber,versionNumber,is_deleted,ID,Name,Aeroexpress,AeroexpressStation,AeroexpressTerminal,AeroexpressTicketWindow,AeroexpressTicketAutomat,AeroexpressWorkingHours,MCDStation,RailwayStation,Latitude_WGS84,Longitude_WGS84,global_id
0,POINT (37.90053 55.41431),62883,None,41,1,0,1,Домодедово,есть,"{'global_id': 1508979300, 'value': 'Аэропорт Д...",есть,есть,есть,"05:30 - 23:30 (с Павелецкого вокзала), 06:00 -...",[],[],55.414307,37.900526,1272818040
1,POINT (37.41571 55.96677),62883,None,41,1,0,2,Шереметьево,есть,"{'global_id': 1508983189, 'value': 'Аэропорт Ш...",есть,есть,есть,"5:15 - 00:31 (с Белорусского вокзала), 05:14 -...",[],[],55.966771,37.415713,1272818309
2,POINT (37.28629 55.60506),62883,None,41,1,0,3,Внуково,нет,[],нет,нет,нет,,[],[],55.605059,37.286287,1272818481
3,POINT (37.51082 55.50286),62883,None,41,1,0,4,Остафьево,нет,[],нет,нет,нет,,[],[],55.502859,37.510824,1272818826
4,POINT (38.11781 55.56155),62883,None,41,1,0,5,Жуковский,нет,[],нет,нет,нет,,[],[],55.561550,38.117812,1272818941


Посмотрим на объекты на карте


In [40]:
dataGDF_final.explore(tiles="cartodb positron")

#### 1.5 Сохраняем данные


In [ ]:
dataGDF_final.to_file('data_gdf.shp')

/var/folders/ry/9bb7wrz54vq_kn2ytlj6ynzm0000gn/T/ipykernel_6053/1833685107.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  buildings.to_file('okn_points.shp')


## 2. Фотографии для объектов на интерактивной карте


In [44]:
df = dataGDF_final

# Папка с фото
photo_folder = "photos"

# Создаём карту
m = folium.Map(location=[55.75, 37.6], zoom_start=9, tiles="cartodb positron")

# Обход строк GeoDataFrame
for _, row in df.iterrows():
    lat = row.geometry.y
    lon = row.geometry.x
    name = row["Name"]
    photo_filename = f"{name}.jpeg"
    photo_path = os.path.join(photo_folder, photo_filename)

    if os.path.exists(photo_path):
        with open(photo_path, "rb") as f:
            img_base64 = base64.b64encode(f.read()).decode("utf-8")
        html = f'<h4>{name}</h4><img src="data:image/jpeg;base64,{img_base64}" width="300">'
    else:
        html = f"<h4>{name}</h4><p><i>Фото не найдено</i></p>"

    iframe = folium.IFrame(html=html, width=310, height=250)
    popup = folium.Popup(iframe, max_width=310)

    folium.Marker(
        location=[lat, lon],
        popup=popup,
        tooltip=name,
        icon=folium.Icon(icon="plane", prefix="fa", color="blue")
    ).add_to(m)

In [45]:
m